### Well, this is non-deteministic random search alghoritm, sometimes works..

### Anybody know alghoritm with polynomial time? Let me know in the comments below)

In [ ]:
import pandas as pd
import numpy as np
import itertools
from warnings import filterwarnings
filterwarnings("ignore")

def calc_spread_return_per_day(df, portfolio_size=200, toprank_weight_ratio=2):
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size=200, toprank_weight_ratio=2):
    buf = df.groupby('Date').apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio, buf

def add_rank(df):
    df["Rank"] = df.groupby("Date")["Target"].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

def find_n(df_dayX, cap=11.4, mse_trashold = 1e-1, n_init=2, n_samples=20,step=1):
    if n_init > 2:
        if (n_init + 200) > 2000:
            n_max = 2000
        else:
            n_max = n_init + 100
    else:
        n_max = 2000
    for n in range(n_init,n_max,step):
        tailnp = np.arange(n, 2000, dtype=int)
        for i in range(n_samples):
            permuted = np.random.permutation(n)
            df_dayX['Rank'] = np.concatenate((permuted, tailnp), axis=None)
            Return = calc_spread_return_per_day(df_dayX)
            cur_mse = (Return - cap)**2
            if cur_mse < mse_trashold:
                return n, permuted, cur_mse
    return -1,-1,-1
            
def v_adjuster(df_dayX, cap=11.4, mse_trashold = 1e-2):
    n, best_ranks, mse_trashold = find_n(df_dayX, cap, mse_trashold, n_samples=10,step=10)
    tmp_n = n
    while tmp_n != -1:
        tmp_n, tmp_best_ranks, tmp_mse = find_n(df_dayX, cap, mse_trashold, n_init=tmp_n, n_samples=250)
        if tmp_n != -1:
            mse_trashold = tmp_mse
            n = tmp_n
            best_ranks = tmp_best_ranks.copy()
    print(mse_trashold)
    
    best_ranks_f = np.concatenate((best_ranks, np.arange(n, 2000, dtype=int)), axis=None)
    
    return best_ranks_f

In [ ]:
df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', parse_dates=["Date"])
df = add_rank(df)
df = df.sort_values(["Date", "Rank"])

In [ ]:
df = df.sort_values(["Date", "Rank"])
for i in range(10):
    for date in df.Date.unique():
        print(date)
        if (calc_spread_return_per_day(df.loc[df.Date==date]) - 11.4)**2 > 1e-11:
            df.loc[df.Date==date, "Rank"] = v_adjuster(df[df.Date==date])
    
sharpe_ratio, buf = calc_spread_return_sharpe(df)
sharpe_ratio

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()
for prices, _, _, _, _, sample_prediction in iter_test:
    ff = df[df['Date']==prices["Date"].iloc[0]]
    mp = ff.set_index("SecuritiesCode")["Rank"]
    sample_prediction["Rank"] = sample_prediction.SecuritiesCode.map(mp)
    env.predict(sample_prediction)